In [178]:
import pandas as pd
import unidecode

# Tratamento final dos arquivos para carga no DW

## Dimensão Pessoa

In [277]:
df_contribuintes = pd.read_csv('dados_prefeitura/receitas_pm_cz/Carga_DW/devedores_carga.csv',sep=';', encoding='utf-8')
df_contribuintes

,Nome,CPF/CNPJ
0,PREF MUNICIPAL DE CAJAZEIRAS,08.923.971/0001-15
1,CAIXA ECONOMICA FEDERAL,00.360.305/0040-10
2,FOPAG,NaN
3,JURAMIR OLIVEIRA DE SOUSA,603.496.544-68
4,PAULA LAIS DE OLIVEIRA SANTANA,064.659.174-67
...,...,...
1187,4HEALTH SERVIÇOS MÉDICOS LTDA,22.571.753/0001-90
1188,ITARGET TECNOLOGIA DA INFORMACAO LTDA,02.130.122/0001-28
1189,INSTITUTO EDUCACIONAL E TECNOLOGICO PRIMEIRA O...,13.770.411/0001-08
1190,GRUPO ESPEC. EM TECNOLOGIA E EXT. COMUNITARIA-...,01.228.887/0001-32


In [278]:
df_contribuintes['Tipo'] = 'Contribuinte'

In [279]:
df_contribuintes['CPF/CNPJ'].isnull().sum()

1

*Temos um item no data frame que é nulo, FOPAG (folha de pagamento). Atribuí o valor de 1 em seu documento.*

In [280]:
df_credores = pd.read_csv('dados_prefeitura/despesas_pm_cz/Arquivos para carga/credores_carga.csv', sep=';', encoding='utf-8')
df_credores

,Nome,CPF/CNPJ,Tipo
0,CAGEPA-COM DE AGUA E ESGOTOS DA PARAIBA,09.123.654/0001-87,Credor
1,Prefeitura Municipal de Cajazeiras,08.923.971/0001-15,Credor
2,BANCO DO BRASIL S.A.,00.000.000/0099-03,Credor
3,PODER JUDICIARIO DO ESTADO DA PARAIBA,09.283.185/0001-63,Credor
4,HENRIQUE SERGIO ALVES DA CUNHA,738.558.394-68,Credor
...,...,...,...
5359,EDNALDO DA SILVA,064.071.334-32,Credor
5360,INSTITUTO DE NEUROLOGIA E NEUROCIRURGIA DA PAR...,03.057.240/0001-10,Credor
5361,FABIO HENRIQUE BATISTA ROLIM,11.593.208/0001-88,Credor
5362,FREITAS SERVICOS DE ENGENHARIA LTDA,26.743.365/0001-08,Credor


In [281]:
df_credores.loc[df_credores['Nome']=='FOPAG']

,Nome,CPF/CNPJ,Tipo
2717,FOPAG,NaN,Credor


In [282]:
df_credores['CPF/CNPJ'].isnull().sum()

1

In [283]:
df_credores['CPF/CNPJ'].fillna(1, inplace=True)

In [284]:
df_credores['CPF/CNPJ'].isnull().sum()

0

In [285]:
df_credores.loc[df_credores['Nome']=='FOPAG']

,Nome,CPF/CNPJ,Tipo
2717,FOPAG,1,Credor


In [286]:
df_join_pessoas = pd.merge(df_contribuintes,df_credores, how='inner', on='CPF/CNPJ')
df_join_pessoas['Tipo'] = 'Contribuinte & Credor'

In [287]:
df_join_pessoas

,Nome_x,CPF/CNPJ,Tipo_x,Nome_y,Tipo_y,Tipo
0,PREF MUNICIPAL DE CAJAZEIRAS,08.923.971/0001-15,Contribuinte,Prefeitura Municipal de Cajazeiras,Credor,Contribuinte & Credor
1,CAIXA ECONOMICA FEDERAL,00.360.305/0040-10,Contribuinte,CAIXA ECONOMICA FEDERAL,Credor,Contribuinte & Credor
2,JURAMIR OLIVEIRA DE SOUSA,603.496.544-68,Contribuinte,JURAMIR OLIVEIRA DE SOUSA,Credor,Contribuinte & Credor
3,PAULA LAIS DE OLIVEIRA SANTANA,064.659.174-67,Contribuinte,PAULA LAIS DE OLIVEIRA SANTANA,Credor,Contribuinte & Credor
4,ANTONIO TOMAZ DE AQUINO,500.464.204-44,Contribuinte,ANTONIO TOMAZ DE AQUINO,Credor,Contribuinte & Credor
...,...,...,...,...,...,...
1139,4HEALTH SERVIÇOS MÉDICOS LTDA,22.571.753/0001-90,Contribuinte,4HEALTH SERVIÇOS MÉDICOS LTDA,Credor,Contribuinte & Credor
1140,ITARGET TECNOLOGIA DA INFORMACAO LTDA,02.130.122/0001-28,Contribuinte,ITARGET TECNOLOGIA DA INFORMACAO LTDA,Credor,Contribuinte & Credor
1141,INSTITUTO EDUCACIONAL E TECNOLOGICO PRIMEIRA O...,13.770.411/0001-08,Contribuinte,INSTITUTO EDUCACIONAL E TECNOLOGICO PRIMEIRA O...,Credor,Contribuinte & Credor
1142,GRUPO ESPEC. EM TECNOLOGIA E EXT. COMUNITARIA-...,01.228.887/0001-32,Contribuinte,GRUPO ESPEC. EM TECNOLOGIA E EXT. COMUNITARIA-...,Credor,Contribuinte & Credor


In [288]:
df_join_pessoas.drop(columns=['Nome_x','Tipo_x','Tipo_y'], inplace=True)
df_join_pessoas.rename(columns={'Nome_y':'Nome'},inplace=True)
df_join_pessoas

,CPF/CNPJ,Nome,Tipo
0,08.923.971/0001-15,Prefeitura Municipal de Cajazeiras,Contribuinte & Credor
1,00.360.305/0040-10,CAIXA ECONOMICA FEDERAL,Contribuinte & Credor
2,603.496.544-68,JURAMIR OLIVEIRA DE SOUSA,Contribuinte & Credor
3,064.659.174-67,PAULA LAIS DE OLIVEIRA SANTANA,Contribuinte & Credor
4,500.464.204-44,ANTONIO TOMAZ DE AQUINO,Contribuinte & Credor
...,...,...,...
1139,22.571.753/0001-90,4HEALTH SERVIÇOS MÉDICOS LTDA,Contribuinte & Credor
1140,02.130.122/0001-28,ITARGET TECNOLOGIA DA INFORMACAO LTDA,Contribuinte & Credor
1141,13.770.411/0001-08,INSTITUTO EDUCACIONAL E TECNOLOGICO PRIMEIRA O...,Contribuinte & Credor
1142,01.228.887/0001-32,GRUPO ESPEC. EM TECNOLOGIA E EXT. COMUNITARIA-...,Contribuinte & Credor


In [289]:
df_carga_pessoas = pd.merge(df_contribuintes,df_credores, how='outer')
df_carga_pessoas

,Nome,CPF/CNPJ,Tipo
0,PREF MUNICIPAL DE CAJAZEIRAS,08.923.971/0001-15,Contribuinte
1,CAIXA ECONOMICA FEDERAL,00.360.305/0040-10,Contribuinte
2,FOPAG,NaN,Contribuinte
3,JURAMIR OLIVEIRA DE SOUSA,603.496.544-68,Contribuinte
4,PAULA LAIS DE OLIVEIRA SANTANA,064.659.174-67,Contribuinte
...,...,...,...
6551,EDNALDO DA SILVA,064.071.334-32,Credor
6552,INSTITUTO DE NEUROLOGIA E NEUROCIRURGIA DA PAR...,03.057.240/0001-10,Credor
6553,FABIO HENRIQUE BATISTA ROLIM,11.593.208/0001-88,Credor
6554,FREITAS SERVICOS DE ENGENHARIA LTDA,26.743.365/0001-08,Credor


In [290]:
df_carga_pessoas.drop_duplicates(subset='CPF/CNPJ', keep='first', inplace=True)
df_carga_pessoas

,Nome,CPF/CNPJ,Tipo
0,PREF MUNICIPAL DE CAJAZEIRAS,08.923.971/0001-15,Contribuinte
1,CAIXA ECONOMICA FEDERAL,00.360.305/0040-10,Contribuinte
2,FOPAG,NaN,Contribuinte
3,JURAMIR OLIVEIRA DE SOUSA,603.496.544-68,Contribuinte
4,PAULA LAIS DE OLIVEIRA SANTANA,064.659.174-67,Contribuinte
...,...,...,...
6550,VINICIUS DE MATOS LEITAO 03692211414,36.114.177/0001-66,Credor
6551,EDNALDO DA SILVA,064.071.334-32,Credor
6552,INSTITUTO DE NEUROLOGIA E NEUROCIRURGIA DA PAR...,03.057.240/0001-10,Credor
6553,FABIO HENRIQUE BATISTA ROLIM,11.593.208/0001-88,Credor


In [291]:
dic_tipo = {'Contribuinte':1,'Credor':2,'Contribuinte & Credor':3}

In [292]:
df_join_pessoas

,CPF/CNPJ,Nome,Tipo
0,08.923.971/0001-15,Prefeitura Municipal de Cajazeiras,Contribuinte & Credor
1,00.360.305/0040-10,CAIXA ECONOMICA FEDERAL,Contribuinte & Credor
2,603.496.544-68,JURAMIR OLIVEIRA DE SOUSA,Contribuinte & Credor
3,064.659.174-67,PAULA LAIS DE OLIVEIRA SANTANA,Contribuinte & Credor
4,500.464.204-44,ANTONIO TOMAZ DE AQUINO,Contribuinte & Credor
...,...,...,...
1139,22.571.753/0001-90,4HEALTH SERVIÇOS MÉDICOS LTDA,Contribuinte & Credor
1140,02.130.122/0001-28,ITARGET TECNOLOGIA DA INFORMACAO LTDA,Contribuinte & Credor
1141,13.770.411/0001-08,INSTITUTO EDUCACIONAL E TECNOLOGICO PRIMEIRA O...,Contribuinte & Credor
1142,01.228.887/0001-32,GRUPO ESPEC. EM TECNOLOGIA E EXT. COMUNITARIA-...,Contribuinte & Credor


In [293]:
df_carga_pessoas = pd.merge(df_carga_pessoas,df_join_pessoas, how='outer')
df_carga_pessoas

,Nome,CPF/CNPJ,Tipo
0,PREF MUNICIPAL DE CAJAZEIRAS,08.923.971/0001-15,Contribuinte
1,CAIXA ECONOMICA FEDERAL,00.360.305/0040-10,Contribuinte
2,FOPAG,NaN,Contribuinte
3,JURAMIR OLIVEIRA DE SOUSA,603.496.544-68,Contribuinte
4,PAULA LAIS DE OLIVEIRA SANTANA,064.659.174-67,Contribuinte
...,...,...,...
6551,4HEALTH SERVIÇOS MÉDICOS LTDA,22.571.753/0001-90,Contribuinte & Credor
6552,ITARGET TECNOLOGIA DA INFORMACAO LTDA,02.130.122/0001-28,Contribuinte & Credor
6553,INSTITUTO EDUCACIONAL E TECNOLOGICO PRIMEIRA O...,13.770.411/0001-08,Contribuinte & Credor
6554,GRUPO ESPEC. EM TECNOLOGIA E EXT. COMUNITARIA-...,01.228.887/0001-32,Contribuinte & Credor


In [294]:
df_carga_pessoas['Cod_Tipo'] = df_carga_pessoas['Tipo'].map(dic_tipo)
df_carga_pessoas

,Nome,CPF/CNPJ,Tipo,Cod_Tipo
0,PREF MUNICIPAL DE CAJAZEIRAS,08.923.971/0001-15,Contribuinte,1
1,CAIXA ECONOMICA FEDERAL,00.360.305/0040-10,Contribuinte,1
2,FOPAG,NaN,Contribuinte,1
3,JURAMIR OLIVEIRA DE SOUSA,603.496.544-68,Contribuinte,1
4,PAULA LAIS DE OLIVEIRA SANTANA,064.659.174-67,Contribuinte,1
...,...,...,...,...
6551,4HEALTH SERVIÇOS MÉDICOS LTDA,22.571.753/0001-90,Contribuinte & Credor,3
6552,ITARGET TECNOLOGIA DA INFORMACAO LTDA,02.130.122/0001-28,Contribuinte & Credor,3
6553,INSTITUTO EDUCACIONAL E TECNOLOGICO PRIMEIRA O...,13.770.411/0001-08,Contribuinte & Credor,3
6554,GRUPO ESPEC. EM TECNOLOGIA E EXT. COMUNITARIA-...,01.228.887/0001-32,Contribuinte & Credor,3


In [295]:
df_carga_pessoas['CPF/CNPJ'].value_counts()

08.923.971/0001-15    2
037.887.234-67        2
100.394.108-70        2
100.011.524-08        2
067.811.084-09        2
                     ..
077.074.494-01        1
080.490.264-00        1
646.603.114-49        1
01.049.344/0001-58    1
26.743.365/0001-08    1
Name: CPF/CNPJ, Length: 5411, dtype: int64

In [296]:
df_carga_pessoas.drop_duplicates(subset='CPF/CNPJ', keep='last', inplace=True)
df_carga_pessoas

,Nome,CPF/CNPJ,Tipo,Cod_Tipo
2,FOPAG,NaN,Contribuinte,1
67,FRANCINERY DA SILVA CAMPOS M.E,13.530.513/0001-56,Contribuinte,1
110,FRANCISCO DO NASCIMENTO CAMPOS M.E.,41.141.599/0001-60,Contribuinte,1
154,BELISARIO JACO DE MORAIS SEGUNDO ME,12.397.179/0001-41,Contribuinte,1
155,BANCO DO BRASIL SA,00.000.000/1641-19,Contribuinte,1
...,...,...,...,...
6551,4HEALTH SERVIÇOS MÉDICOS LTDA,22.571.753/0001-90,Contribuinte & Credor,3
6552,ITARGET TECNOLOGIA DA INFORMACAO LTDA,02.130.122/0001-28,Contribuinte & Credor,3
6553,INSTITUTO EDUCACIONAL E TECNOLOGICO PRIMEIRA O...,13.770.411/0001-08,Contribuinte & Credor,3
6554,GRUPO ESPEC. EM TECNOLOGIA E EXT. COMUNITARIA-...,01.228.887/0001-32,Contribuinte & Credor,3


In [297]:
df_carga_pessoas['Cod_Tipo'].value_counts()

2    4220
3    1144
1      48
Name: Cod_Tipo, dtype: int64

In [298]:
def formatar_palavras(pessoa):
    processamento = unidecode.unidecode(pessoa.lower())
    if len(processamento) > 50:
        return processamento[0:49]
    return processamento

In [299]:
df_carga_pessoas['Nome'] = df_carga_pessoas['Nome'].apply(formatar_palavras)

In [300]:
df_carga_pessoas

,Nome,CPF/CNPJ,Tipo,Cod_Tipo
2,fopag,NaN,Contribuinte,1
67,francinery da silva campos m.e,13.530.513/0001-56,Contribuinte,1
110,francisco do nascimento campos m.e.,41.141.599/0001-60,Contribuinte,1
154,belisario jaco de morais segundo me,12.397.179/0001-41,Contribuinte,1
155,banco do brasil sa,00.000.000/1641-19,Contribuinte,1
...,...,...,...,...
6551,4health servicos medicos ltda,22.571.753/0001-90,Contribuinte & Credor,3
6552,itarget tecnologia da informacao ltda,02.130.122/0001-28,Contribuinte & Credor,3
6553,instituto educacional e tecnologico primeira opca,13.770.411/0001-08,Contribuinte & Credor,3
6554,grupo espec. em tecnologia e ext. comunitaria- ge,01.228.887/0001-32,Contribuinte & Credor,3


In [301]:
df_carga_pessoas.dtypes

Nome        object
CPF/CNPJ    object
Tipo        object
Cod_Tipo     int64
dtype: object

In [309]:
df_carga_pessoas['CPF/CNPJ'].isnull().sum()

1

In [302]:
df_carga_pessoas.to_csv('dados_prefeitura/Arquivos para carga final/carga_pessoas.csv', index=None, sep=';', )
df_carga_pessoas.to_csv('dados_prefeitura/Arquivos para carga final/carga_pessoas.txt', index=None, sep=';', )

## Dimensão Tempo

In [2]:
df_tempo_receitas = pd.read_csv('dados_prefeitura/receitas_pm_cz/Carga_DW/tempo_carga.csv', sep=';')
df_tempo_receitas

,Data,Ano,Mes,Dia,Trimestre,Cod_Trimestre,Semestre,Cod_Semestre,Cod_Mes
0,2013-01-02,2013,janeiro,2,Primeiro Trimestre,1,Primeiro Semestre,1,1
1,2013-01-03,2013,janeiro,3,Primeiro Trimestre,1,Primeiro Semestre,1,1
2,2013-01-04,2013,janeiro,4,Primeiro Trimestre,1,Primeiro Semestre,1,1
3,2013-01-07,2013,janeiro,7,Primeiro Trimestre,1,Primeiro Semestre,1,1
4,2013-01-08,2013,janeiro,8,Primeiro Trimestre,1,Primeiro Semestre,1,1
...,...,...,...,...,...,...,...,...,...
2071,2020-12-24,2020,dezembro,24,Quarto Trimestre,4,Segundo Semestre,2,12
2072,2020-12-28,2020,dezembro,28,Quarto Trimestre,4,Segundo Semestre,2,12
2073,2020-12-29,2020,dezembro,29,Quarto Trimestre,4,Segundo Semestre,2,12
2074,2020-12-30,2020,dezembro,30,Quarto Trimestre,4,Segundo Semestre,2,12


In [3]:
df_tempo_despesas = pd.read_csv('dados_prefeitura/despesas_pm_cz/Arquivos para carga/tempo_carga.csv', sep=';')
df_tempo_despesas

,Data,Ano,Mes,Dia,Trimestre,Cod_Trimestre,Semestre,Cod_Semestre,Cod_Mes
0,2013-01-02,2013,janeiro,2,Primeiro Trimestre,1,Primeiro Semestre,1,1
1,2013-01-03,2013,janeiro,3,Primeiro Trimestre,1,Primeiro Semestre,1,1
2,2013-01-23,2013,janeiro,23,Primeiro Trimestre,1,Primeiro Semestre,1,1
3,2013-01-29,2013,janeiro,29,Primeiro Trimestre,1,Primeiro Semestre,1,1
4,2013-01-30,2013,janeiro,30,Primeiro Trimestre,1,Primeiro Semestre,1,1
...,...,...,...,...,...,...,...,...,...
1789,2020-12-21,2020,dezembro,21,Quarto Trimestre,4,Segundo Semestre,2,12
1790,2020-12-22,2020,dezembro,22,Quarto Trimestre,4,Segundo Semestre,2,12
1791,2020-12-24,2020,dezembro,24,Quarto Trimestre,4,Segundo Semestre,2,12
1792,2020-12-28,2020,dezembro,28,Quarto Trimestre,4,Segundo Semestre,2,12


In [4]:
df_carga_tempo

,Data,Ano,Mes,Dia,Trimestre,Cod_Trimestre,Semestre,Cod_Semestre,Cod_Mes
0,2013-01-02,2013,janeiro,2,Primeiro Trimestre,1,Primeiro Semestre,1,1
1,2013-01-03,2013,janeiro,3,Primeiro Trimestre,1,Primeiro Semestre,1,1
2,2013-01-04,2013,janeiro,4,Primeiro Trimestre,1,Primeiro Semestre,1,1
3,2013-01-07,2013,janeiro,7,Primeiro Trimestre,1,Primeiro Semestre,1,1
4,2013-01-08,2013,janeiro,8,Primeiro Trimestre,1,Primeiro Semestre,1,1
...,...,...,...,...,...,...,...,...,...
2084,2014-08-23,2014,agosto,23,Terceiro Trimestre,3,Segundo Semestre,2,8
2085,2015-01-24,2015,janeiro,24,Primeiro Trimestre,1,Primeiro Semestre,1,1
2086,2016-01-23,2016,janeiro,23,Primeiro Trimestre,1,Primeiro Semestre,1,1
2087,2017-03-19,2017,marco,19,Primeiro Trimestre,1,Primeiro Semestre,1,3


In [5]:
df_join_tempo.to_csv('dados_prefeitura/Arquivos para carga final/carga_tempo.csv', index=None, sep=';')
df_join_tempo.to_csv('dados_prefeitura/Arquivos para carga final/carga_tempo.txt', index=None, sep=';')

## Dimensão Tipo Receita

In [263]:
df_tipo_receitas = pd.read_csv('dados_prefeitura/receitas_pm_cz/Carga_DW/tipo_carga.csv', sep=';')
df_tipo_receitas

,Código,Receita
0,1722010101,COTA-PARTE DO ICMS
1,1721010201,COTA-PARTE DO FUNDO DE PARTICIPAÇÃO DOS MUNICI...
2,9722010101,DEDUÇÃO DE RECEITA PARA A FORMAÇÃO DO FUNDEF/ ...
3,9721010201,DEDUÇÃO DE RECEITA DO FPM/ FUNDEF E REDUTOR FI...
4,1113050002,SNA - SIMPLES NACIONAL
...,...,...
237,1718991104,Outras Transferências da União - LC 173/2020 A...
238,2418109102,Transferências de Convênio da União - Ministér...
239,1718991105,Outras Transferências da União  Lei Aldir Blanc
240,1121041101,Taxa de Controle e Fiscalização Ambiental


In [264]:
df_tipo_receitas['Receita'] = df_tipo_receitas['Receita'].apply(formatar_palavras)
df_tipo_receitas

,Código,Receita
0,1722010101,cota-parte do icms
1,1721010201,cota-parte do fundo de participacao dos munici...
2,9722010101,deducao de receita para a formacao do fundef/ ...
3,9721010201,deducao de receita do fpm/ fundef e redutor finan
4,1113050002,sna - simples nacional
...,...,...
237,1718991104,outras transferencias da uniao - lc 173/2020 art
238,2418109102,transferencias de convenio da uniao - ministerio
239,1718991105,outras transferencias da uniao lei aldir blanc
240,1121041101,taxa de controle e fiscalizacao ambiental


In [265]:
df_tipo_receitas.drop_duplicates(subset='Código', keep='first', inplace=True)

In [266]:
df_tipo_receitas

,Código,Receita
0,1722010101,cota-parte do icms
1,1721010201,cota-parte do fundo de participacao dos munici...
2,9722010101,deducao de receita para a formacao do fundef/ ...
3,9721010201,deducao de receita do fpm/ fundef e redutor finan
4,1113050002,sna - simples nacional
...,...,...
237,1718991104,outras transferencias da uniao - lc 173/2020 art
238,2418109102,transferencias de convenio da uniao - ministerio
239,1718991105,outras transferencias da uniao lei aldir blanc
240,1121041101,taxa de controle e fiscalizacao ambiental


In [267]:
df_tipo_receitas.to_csv('dados_prefeitura/Arquivos para carga final/carga_tipo.csv', index=None, sep=';', )
df_tipo_receitas.to_csv('dados_prefeitura/Arquivos para carga final/carga_tipo.txt', index=None, sep=';', )

## Dimensão Receita

In [310]:
df_carga_receita = pd.read_csv('dados_prefeitura/receitas_pm_cz/Carga_DW/receita_carga.csv', sep=';')
df_carga_receita

,Número Guia,Valor Previsto,Valor Lançado,Valor Arrecadado,CPF/CNPJ,Data,Código
0,75,2700000.0,0,0.00,08.923.971/0001-15,2013-01-02,1113050001
1,222,2700000.0,0,82.38,08.923.971/0001-15,2013-01-02,1113050001
2,76,2700000.0,0,0.00,08.923.971/0001-15,2013-01-02,1113050001
3,74,2700000.0,0,0.00,08.923.971/0001-15,2013-01-02,1113050001
4,73,2700000.0,0,0.00,08.923.971/0001-15,2013-01-02,1113050001
...,...,...,...,...,...,...,...
78128,9066,53000.0,0,0.18,00.000.000/0099-03,2020-12-31,1321001101
78129,9065,53000.0,0,2.62,00.000.000/0099-03,2020-12-31,1321001101
78130,9064,430000.0,0,172.00,12.428.243/0001-04,2020-12-31,1220991100
78131,9077,53000.0,0,2.07,00.000.000/0099-03,2020-12-31,1321001101


In [312]:
df_carga_receita['CPF/CNPJ'].value_counts().sum()

70422

In [313]:
df_carga_receita['CPF/CNPJ'].fillna(value=1, inplace=True)

In [314]:
df_carga_receita['CPF/CNPJ'].value_counts().sum()

78133

In [321]:
df_carga_receita.to_csv('dados_prefeitura/Arquivos para carga final/carga_receita.csv', index=None, sep=';', )
df_carga_receita.to_csv('dados_prefeitura/Arquivos para carga final/carga_receita.txt', index=None, sep=';', )

## Dimensão Despesa

In [316]:
df_2013 = pd.read_csv('dados_prefeitura/despesas_pm_cz/2013/02_despesas/despesas_2013_tratado.csv', sep=';')
df_2014 = pd.read_csv('dados_prefeitura/despesas_pm_cz/2014/02_despesas/despesas_2014_tratado.csv', sep=';')
df_2015 = pd.read_csv('dados_prefeitura/despesas_pm_cz/2015/02_despesas/despesas_2015_tratado.csv', sep=';')
df_2016 = pd.read_csv('dados_prefeitura/despesas_pm_cz/2016/02_despesas/despesas_2016_tratado.csv', sep=';')
df_2017 = pd.read_csv('dados_prefeitura/despesas_pm_cz/2017/02_despesas/despesas_2017_tratado.csv', sep=';')
df_2018 = pd.read_csv('dados_prefeitura/despesas_pm_cz/2018/02_despesas/despesas_2018_tratado.csv', sep=';')
df_2019 = pd.read_csv('dados_prefeitura/despesas_pm_cz/2019/02_despesas/despesas_2019_tratado.csv', sep=';')
df_2020 = pd.read_csv('dados_prefeitura/despesas_pm_cz/2020/02_despesas/despesas_2020_tratado.csv', sep=';')

In [317]:
df_credores_carga = pd.merge(df_2013,df_2014, how='outer')
df_credores_carga = pd.merge(df_credores_carga, df_2015, how = 'outer')
df_credores_carga = pd.merge(df_credores_carga, df_2016, how = 'outer')
df_credores_carga = pd.merge(df_credores_carga, df_2017, how = 'outer')
df_credores_carga = pd.merge(df_credores_carga, df_2018, how = 'outer')
df_credores_carga = pd.merge(df_credores_carga, df_2019, how = 'outer')
df_credores_carga = pd.merge(df_credores_carga, df_2020, how = 'outer')
df_credores_carga

,NÃºmero,Empenhado(R$),Liquidado(R$),Pago(R$),Saldo(R$),Despesa,CPF/CNPJ,Data
0,5,45.68,45.68,45.68,0.00,42,09.123.654/0001-87,2013-01-03
1,17,13309.74,13309.74,13309.74,0.00,38,08.923.971/0001-15,2013-01-30
2,68,37073.10,37073.10,37073.10,0.00,43,00.000.000/0099-03,2013-01-30
3,69,3223.75,3223.75,3223.75,0.00,43,00.000.000/0099-03,2013-01-30
4,70,39698.37,39698.37,39698.37,0.00,43,00.000.000/0099-03,2013-01-30
...,...,...,...,...,...,...,...,...
56184,7527,12045.50,12045.50,0.00,12045.50,714,08.923.971/0001-15,2020-12-31
56185,7558,38292.84,0.00,0.00,38292.84,719,30.733.130/0001-02,2020-12-31
56186,7559,8000.00,0.00,0.00,8000.00,681,30.733.130/0001-02,2020-12-31
56187,7570,33162.95,0.00,0.00,33162.95,719,30.733.130/0001-02,2020-12-31


In [318]:
df_credores_carga['CPF/CNPJ'].isnull().sum()

0

In [320]:
df_credores_carga.to_csv('dados_prefeitura/Arquivos para carga final/carga_despesa.csv', index=None, sep=';', )
df_credores_carga.to_csv('dados_prefeitura/Arquivos para carga final/carga_despesa.txt', index=None, sep=';', )